## Написать приложение или функцию, которые собирают основные новости с сайта на выбор lenta.ru, yandex-новости. Для парсинга использовать XPath.

### Структура данных в виде словаря должна содержать:

- *название источника;

- наименование новости;

- ссылку на новость;

- дата публикации.

Минимум один сайт, максимум - все два

In [1]:
import requests
from pprint import pprint
from lxml import html
import json

# 1)

### https://yandex.ru/news

В Московской области странице Яндекс-новостей 13 категорий: **'top-heading', 'Москва и область0', 'Интересное1', 'Политика2', 'Общество3', 'Экономика4', 'В мире5', 'Спорт6', 'Происшествия7', 'Культура8', 'Технологии9', 'Наука10', 'Авто11'**

и 3 размера блоков новстей:

- *блок 8* - самый большой
- *блок 4*  - среднего размера
- *блок 6* - самый маленький (это, по сути, размер болока 8 с четыремя новостями в нём, т.е. 6+6+6+6=8)

на странице всего 65 блоков разного размера из разных категорий

Написал функцию yandex_news_scraping(), которая собирает и сортирует новости по своим тематическим категориям, пронумеровывая новости в зависимости от их категории. Костяк структуры такой:

$главный \; первый \; раздел$

$\;\;$ -блок-8: general-новость top-heading

$\;\;\;\;\;\;\;\;$ -блок-4: 1-я новость top-heading

$\;\;\;\;\;\;\;\;$ -блок-4: 2-я новость top-heading

$\;\;\;\;\;\;\;\;$ -блок-4: 3-я новость top-heading

$\;\;\;\;\;\;\;\;$ -блок-4: 4-я новость top-heading

$остальные\;разделы$

$\;\;\;\;$ -блок-4: general-новость

$\;\;\;\;\;\;\;\;\;\;$ -блок-6: 1-я новость

$\;\;\;\;\;\;\;\;\;\;$ -блок-6: 2-я новость

$\;\;\;\;\;\;\;\;\;\;$ -блок-6: 3-я новость

$\;\;\;\;\;\;\;\;\;\;$ -блок-6: 4-я новость

каждая новость содержит ключи: 

- **eadlines** - заголовок

- **text_news** - краткое описание

- **link_to_news** - ссылка на новость

- **news_source** - источник новости

- **news_time** - время публикации новости

В итоге мы получаем JSON-структуру данных, которую записываем в файл

P.S: часть путей в xpath может показаться дублированной, но это не совсем так, т.к. отсчсёт пути идёт от названия категории, которое меняется в цикле + такой вид, на мой взгляд, уобен при незначительных изменениях вёрстки на сайте, так как, видя путь, его будет легче подогать под возможные изменения

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
url = 'https://yandex.ru/news/'

session = requests.Session()
response = session.get(url, headers=headers)

print(response.ok) 
dom_yandex = html.fromstring(response.text)

True


In [3]:
def yandex_news_scraping():
    yandex_news = {}
    
    def assemble_a_dictionary(n, i, section_name, category_name, headlines, text_news, link_to_news, news_source, news_time):
        section_name = i
        str_news = f'{n+1} {section_name} {category_name}'
        yandex_news[str_news] = {
            'headlines': headlines,
            'text_news': text_news,
            'link_to_news': link_to_news,
            'news_source': news_source,
            'news_time': news_time}
               
    section = dom_yandex.xpath("//div[@class='mg-grid__col mg-grid__col_xs_12 mg-grid__col_sm_9']/section/@*")
    col_xs_ = [4, 8]
    for n, i in enumerate(section):
        for col in col_xs_:
            section_name = None
            if section.index(i) == 0:
                if col == 8:                
                    str_path = f"//section[@aria-labelledby='{i}']/div/div[contains(@class, 'col_xs_{col}')]/div/div[2]"
                    headlines = ''.join(dom_yandex.xpath(f"{str_path}/h2/a/text()"))
                    text_news = ''.join(dom_yandex.xpath(f"{str_path}/div/text()"))
                    link_to_news = ''.join(dom_yandex.xpath(f"{str_path}/h2/a/@href"))
                    news_source = ''.join(dom_yandex.xpath(f"{str_path}/div[2]/div/div/span/a/text()"))
                    news_time = ''.join(dom_yandex.xpath(f"{str_path}/div[2]/div/div/span[2]/text()"))
                    assemble_a_dictionary(n, i, section_name, 'general', headlines, text_news, link_to_news, news_source, news_time)
                else:
                    str_path = dom_yandex.xpath(f"//section[@aria-labelledby='{i}']/div/div[contains(@class, 'col_xs_{col}')]/div")
                    step = 1
                    for num in str_path:
                        headlines = num.xpath("./div/div/h2/a/text()")[0]
                        text_news = num.xpath("./div/div/div[@class='mg-card__annotation']/text()")[0]
                        link_to_news = num.xpath("./div/div/h2/a/@href")[0]
                        news_source = num.xpath("./div[3]/div/div/span/a/text()")[0]
                        news_time = num.xpath("./div[3]/div/div/span[2]/text()")[0]
                        assemble_a_dictionary(n, i, section_name, step, headlines, text_news, link_to_news, news_source, news_time)
                        step += 1
            else:
                if col == 4:
                    str_path = dom_yandex.xpath(f"//section[@aria-labelledby='{i}']/div[2]/div[contains(@class, 'col_xs_{col}')]/div")
                    step = 1
                    for num in str_path:
                        headlines = num.xpath("./div/div/h2/a/text()")[0]
                        text_news = num.xpath("./div//div/div[@class='mg-card__annotation']/text()")[0]
                        link_to_news = num.xpath("./div/div/h2/a/@href")[0]
                        news_source = num.xpath("./div[3]/div/div/span/a/text()")[0]
                        news_time = num.xpath("./div[3]/div/div/span[2]/text()")[0]
                        assemble_a_dictionary(n, i, section_name, 'big', headlines, text_news, link_to_news, news_source, news_time)
                        step += 1
                else:
                    str_path = dom_yandex.xpath(f"//section[@aria-labelledby='{i}']/div[2]/div[contains(@class, 'col_xs_{col}')]/div/div")
                    step = 1
                    for num in str_path:
                        headlines = num.xpath("./div/div/div/div/h2/a/text()")[0]
                        text_news = num.xpath("./div/div/div/div/div/text()")[0]
                        link_to_news = num.xpath("./div/div/div/div/h2/a/@href")[0]
                        news_source = num.xpath("./div/div[2]/div/div/span/a/text()")[0]
                        news_time = num.xpath("./div/div[2]/div/div/span[2]/text()")[0]
                        assemble_a_dictionary(n, i, section_name, step, headlines, text_news, link_to_news, news_source, news_time)
                        step += 1  
    with open('yandex_news.json', 'w', encoding='UTF-8') as f:
        json.dump(yandex_news, f)
        print('файл "yandex_news.json" создан!')
    return yandex_news

проверим за собой:

In [4]:
yandex = yandex_news_scraping()
print(f'\nколичество новостей: {len(yandex)}')

файл "yandex_news.json" создан!

количество новостей: 65


In [5]:
# выборочно посмотрим на 2 новости, чтоб увидеть вид их записи

items = list(yandex.items())
pprint(items[1])
pprint(items[64])

('1 top-heading 2',
 {'headlines': 'МИД России сообщил о\xa0гибели двух сотрудников посольства '
               'в\xa0Афганистане в\xa0результате теракта',
  'link_to_news': 'https://yandex.ru/news/story/MID_Rossii_soobshhil_ogibeli_dvukh_sotrudnikov_posolstva_vAfganistane_vrezultate_terakta--b1d08d42f95d11e72d64aff14de8b062?lang=ru&rubric=index&fan=1&stid=vYrzegt-3zVIJcJC5hrs&t=1662383735&tt=true&persistent_id=221692774&story=357b5eae-af08-50c7-9802-2055ce4c95bd',
  'news_source': 'Газета.Ru',
  'news_time': '15:42',
  'text_news': 'МИД России сообщил о гибели двух сотрудников посольства в '
               'Афганистане в результате теракта, сообщается на сайте '
               'ведомства.'})
('13 Авто11 4',
 {'headlines': 'Автоэксперт Попов спрогнозировал возврат ушедших с\xa0рынка РФ '
               'автобрендов в\xa02023 году',
  'link_to_news': 'https://yandex.ru/news/story/Avtoehkspert_Popov_sprognoziroval_vozvrat_ushedshikh_srynka_RF_avtobrendov_v2023_godu--284791b53c482d9547362

In [6]:
# # если очень хочется - посмотреть все новости в блокноте можно здесь:
# pprint(yandex)

# 2)

### https://lenta.ru/

На Ленте новости, условно, пожно поделить на две категори, поэтому я написал функцию lenta_news_scraping(), которая принимает 2 аргумента на два типа новостей с сайта lenta.ru:

1) **агумент а = новости, у которых в есть атрибут класса "card-mini".** Таких новостей на сайте самое большое количество, порядка 90. Они, в свою очередь, также размещаются в отдельных тематических блоках, один из которых называется "Главные новости". Для блока "Главных новостей" нет времени публикации и я его выделяю в отдельную категорию

2) **аргумент b  = новости, у которых в есть атрибут класса "card-big".** Эти новости размазанны по всей странице, они сопровождаются картинками и своими заголовками больше напоминают бульварные жёлтые статьи, чем что-то серьёзное. Однако это тоже новости и их на сайте порядка 50-ти (в разное время суток разное количество)

Функция, в зависимости от количества переданных в неё аргументов (1 ли 2) может возвращать на выбор один из типов новостей либо оба типа новостей вместе

Функция возвращает словарь, с вложенным в него словарями новостей. Ключи основного словаря - это пронумерованные категории новостей. Категории такие:

**новости агрумента a:**

$\;\;\;\;\;$ - **regular_news** - регулярные новости

$\;\;\;\;\;$ - **prime_news** - новости из блока "Главные новости"

**новости агрумента b:**

$\;\;\;\;\;$ - **card_big_news** - новости "с картинками" на сайте

$\;\;\;\;\;$ - **новость из раздела с одним большим фото**

Все категории содержат в себе:

- **title** - название новости 

- **link** - кликабельную ссылку на неё (ссылки собирал в абсолютные, т.к. в зависимости от источника они были разными) и

- **time** дату/время публикации ( формат даты/времени у каждого типа новостей разный)

В итоге мы получаем JSON-структуру данных, которую записываем в файл

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
url = 'https://lenta.ru/'

session = requests.Session()
response = session.get(url, headers=headers)

print(response.ok)
dom_lenta = html.fromstring(response.text) 

True


In [8]:
def lenta_news_scraping(a=None, b=None):
    
    url = 'https://lenta.ru/'       
    mini_news = dom_lenta.xpath("//div[@class='card-mini__text']")
    card_big = dom_lenta.xpath("//a[contains(@class, 'card-big')]")
    lenta_list = {}
    
    def filling_json(href, link, time, news_category, n, name_news):
        for x in href:
            if x.find('http'):
                x = url[:-1] + x
            link = x
            lenta_list[f'{n+1} {news_category}'] = {
                'title': name_news,
                'time': time,
                'link': link}

    def regular_news():
        for n, i in enumerate(mini_news):
            news_category = 'regular_news'
            link = None
            name_news = i.xpath("./span[@class='card-mini__title']/text()")[0]
            time = ''.join(i.xpath("./div[@class='card-mini__info']/time/text()"))
            href = i.xpath("./../@href")
            if not time:
                news_category = 'prime_news'
            filling_json(href, link, time, news_category, n, name_news)

    def card_big_news():
        news_category = 'card_big_news'
        for n, i in enumerate(card_big):
            link = None
            name_news =  ''.join(i.xpath("./div[2]/h3/text()"))
            time =  ''.join(i.xpath("./div[3]/time/text()"))
            href = i.xpath("./@href")
            filling_json(href, link, time, news_category, n, name_news)
            
        big_news = dom_lenta.xpath("//a[contains(@class, 'card-feature')]/div[2]/h3/text()")[0]
        big_text = ''.join(dom_lenta.xpath("//span[contains(@class, 'card-feature__description')]/text()"))
        big_href = dom_lenta.xpath("//a[contains(@class, 'card-feature')]/@href")
        big_href = url[:-1] + big_href[0]
        lenta_list[f'1 news with photo'] = {
                'title': big_news + big_text,
                'time': '',
                'link': big_href}
    
    if a:
        regular_news()
    if b:
        card_big_news()
        
    with open('lenta_news.json', 'w', encoding='UTF-8') as f:
        json.dump(lenta_list, f)
        print('файл "lenta_news.json" создан!')
        
    return lenta_list

проверим за собой:

In [9]:
a, b = 1, 2
lenta = lenta_news_scraping(a, b)
print(f'\nколичество новостей: {len(lenta)}')

файл "lenta_news.json" создан!

количество новостей: 143


In [10]:
# выборочно посмотрим на 3 новости, чтоб увидеть вид их записи

items = list(lenta.items())
pprint(items[0])
pprint(items[-1])
pprint(items[105])

('1 regular_news',
 {'link': 'https://lenta.ru/news/2022/09/05/odnoklassniki/',
  'time': '16:21',
  'title': '«Одноклассники» запустили движение благодарностей учителям'})
('1 news with photo',
 {'link': 'https://lenta.ru/articles/2022/09/05/liz_truss/',
  'time': '',
  'title': 'Железная леди — 2. Кто такая новый премьер-министр Великобритании '
           'Лиз Трасс и почему ее не любят в России?'})
('17 card_big_news',
 {'link': 'https://lenta.ru/articles/2022/08/29/porty1/',
  'time': '14:37, 29 августа 2022',
  'title': 'Разворот к морю.'})


In [11]:
# # если очень хочется - посмотреть все новости в блокноте можно здесь:
# pprint(lenta)